In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import CCAtools
import seaborn as sns
from scipy.stats import shapiro
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [2]:
path2PermCCA = "PermCCA/"  ### https://github.com/andersonwinkler/PermCCA
### note environment requirement being phased out ^^
palmPath = "palm-alpha119_2/"  ### path to palm
### see https://github.com/andersonwinkler/PALM

In [ ]:
import matlab.engine as mlab_eng
import matlab

eng = mlab_eng.start_matlab()
eng.addpath(path2PermCCA)
eng.addpath(palmPath)

In [4]:
from CCAtools.preprocessing import (
    prep_confounds,
    cube_root,
    gauss_SM,
    zscore,
    normal_eqn_python,
)

### Load data

In [5]:
### set paths
restricted_hcp_path = ""  ## set path to your copy of hcp restricted data
hcp_behavior_data = ""  ### set path to your copy of hcp behvaioral data
data_dir = ""  ### set to directory for data used in/throughout analysis

In [6]:
#### load behavioral data
### this is HCP restricted data. for access see
### https://www.humanconnectome.org/study/hcp-young-adult/document/restricted-data-usage
RestrictedData = pd.read_csv(restricted_hcp_path, index_col="Subject")
### load non-restricted hcp behavioral data
BehData = pd.read_csv(hcp_behavior_data, index_col="Subject")
### merge the dtaframes
fullData = BehData.merge(RestrictedData, on="Subject")
fullData.index = fullData.index.map(str)
fullData.index = fullData.index.map(str)
### load in square_root of total hemisphere surface area for each subject
Larea = pd.read_csv(f"{data_dir}/LareaFactors.csv")
Larea.rename(index={0: "Larea"}, inplace=True)
Rarea = pd.read_csv(f"{data_dir}/RareaFactors.csv")
Rarea.rename(index={0: "Rarea"}, inplace=True)
area = pd.concat([Larea, Rarea]).T
area.index.names = ["Subject"]
fullData = area.join(fullData, on="Subject", how="inner")

In [7]:
#### Load Distances
LSchaeferDist = pd.read_csv(f"{data_dir}/Schaefer400Geodesic.L.mat.csv")
LSchaeferDist.rename(columns={"Unnamed: 0": "Subject"}, inplace=True)
LSchaeferDist.set_index("Subject", inplace=True)
LSchaeferDist.columns = [f"L.{i}" for i in LSchaeferDist.columns]

RSchaeferDist = pd.read_csv(f"{data_dir}/Schaefer400Geodesic.R.mat.csv")
RSchaeferDist.rename(columns={"Unnamed: 0": "Subject"}, inplace=True)
RSchaeferDist.set_index("Subject", inplace=True)
RSchaeferDist.columns = [f"R.{i}" for i in RSchaeferDist.columns]
SchaeferDist = pd.concat([LSchaeferDist, RSchaeferDist], axis=1)
SchaeferDist.index = SchaeferDist.index.map(str)

In [8]:
#### Load FC data
LSchaeferFC = pd.read_csv(f"{data_dir}/Schaefer200FConn.L.mat.csv")
LSchaeferFC.rename(columns={"Unnamed: 0": "Subject"}, inplace=True)
LSchaeferFC.set_index("Subject", inplace=True)
LSchaeferFC.columns = [f"L.{i}" for i in LSchaeferFC.columns]

RSchaeferFC = pd.read_csv(f"{data_dir}/Schaefer200FConn.R.mat.csv")
RSchaeferFC.rename(columns={"Unnamed: 0": "Subject"}, inplace=True)
RSchaeferFC.set_index("Subject", inplace=True)
RSchaeferFC.columns = [f"R.{i}" for i in RSchaeferFC.columns]
SchaeferFC = pd.concat([LSchaeferFC, RSchaeferFC], axis=1)
SchaeferFC.index = SchaeferFC.index.map(str)

## Define preprocessing functions

In [9]:
##### preprocessing functions we'll add to a package later
from CCAtools.preprocessing import zscore


def prep_confounds_local(confs, eng):
    """set the confounds up with gaussianization and normalization as done by smith et al 2015."""
    assert ("palm" in eng.path()) == True, "add PermCCA to your matlab path"
    mat_data = matlab.double(
        confs.values.tolist()
    )  ### they actually included the binary acquisition data in the gaussianization
    #     print('gaussianizing')
    gaussed = np.asarray(eng.palm_inormal(mat_data))
    squared = gaussed[:, 1:] ** 2
    ready_confs = np.hstack([gaussed, squared])
    ready_confs = zscore(np.hstack([gaussed, squared]), ax=0)

    return ready_confs


def set_confounds(data, mlab_eng):
    """takes in a full data set of all HCP variables and extracts and preprocesses confounds to be regressed"""
    eng = mlab_eng
    confounds = data
    gend = LabelEncoder().fit_transform(confounds["Gender"])
    acq = LabelEncoder().fit_transform(confounds["Acquisition"])
    acq[acq < 2] = 0
    acq[acq > 0] = 1
    df = confounds.copy()
    df["Acquisition"] = acq
    df["Gender"] = gend
    df["FS_IntraCranial_Vol"] = data["FS_IntraCranial_Vol"].map(cube_root)
    df["FS_BrainSeg_Vol"] = data["FS_BrainSeg_Vol"].map(cube_root)
    df["Larea"] = data["Larea"].map(np.sqrt)
    df["Rarea"] = data["Rarea"].map(np.sqrt)
    confounds = prep_confounds_local(df, eng)
    return confounds


def preprocess_SM(data, confs, mlab_eng):
    """preprocess the subject measures. Guassianize and remove confounds."""
    eng = mlab_eng
    assert ("palm" in eng.path()) == True, "add PermCCA to your matlab path"
    data = data
    gaussed = gauss_SM(data, eng)
    residuals = normal_eqn_python(confs, gaussed)
    cleaned = zscore(residuals)
    cleaned = pd.DataFrame(cleaned, index=data.index, columns=data.columns)
    return cleaned


def preprocessDists(data, confounds):
    NET = data.copy()
    dims = NET.shape
    ##### check for vertices with no variance i.e guaranteed masks
    steady_masks = np.where(np.sum(NET) == 0)[0]
    valididx = np.where(np.sum(NET) != 0)[0]

    if len(steady_masks) != 0:
        NET = NET.iloc[:, valididx]

    #     amNET = np.abs(np.nanmean(NET, axis=0))
    NET1 = NET  # /amNET
    NET1 = NET1 - np.mean(NET1, axis=0)
    NET1 = NET1 / np.nanstd(NET1.values.flatten())
    NET1 = normal_eqn_python(confounds, NET1)
    NET1 = pd.DataFrame(NET1, columns=NET.columns, index=data.index)

    if len(steady_masks) != 0:
        out = np.zeros(dims)
        out[:, valididx] = NET1.values
        NET1 = pd.DataFrame(out, index=NET.index)

    return NET1


def clean_data(subjectList, all_confs, all_SM, all_dist, mlab=eng):
    ### remove confounds for a group of subjects
    ### always done independently to avoid leakage

    ## set the confounds
    confs = all_confs.loc[subjectList]
    confs = set_confounds(confs, mlab)
    ## regress them from behavioral measures
    behavior = all_SM.loc[subjectList]
    behavior = preprocess_SM(behavior, confs, mlab)
    ## regress them from distance measures
    distance = all_dist.loc[subjectList]
    distance = preprocessDists(distance, confs)

    return confs, behavior, distance

#### Extract confounds and behaviors of interest from raw data

In [10]:
conf_categories = fullData[
    [
        "Acquisition",
        "Gender",
        "Age_in_Yrs",
        "Height",
        "Weight",
        "BPSystolic",
        "BPDiastolic",
        "FS_IntraCranial_Vol",
        "FS_BrainSeg_Vol",
        "Larea",
        "Rarea",
    ]
]
# conf_categories.dropna(inplace=True)

SensoryTasks = ["ReadEng_Unadj"]
# subjects=list(fullData.index)
subjSM = fullData[SensoryTasks]


full_subjList = conf_categories.merge(subjSM, on="Subject").dropna().index
full_subjList.to_series().to_csv("SubjectInclusionList.csv")

conf_categories = conf_categories.loc[full_subjList]
subjSM = subjSM.loc[full_subjList]

complete_data = fullData.loc[full_subjList]

### Load prebaked cross validation as in feature selection

In [11]:
import json

# Opening JSON file
f = open("cross_validation_folds.json")

# returns JSON object as
# a dictionary
folds = json.load(f)

### Load positive and negative distances associated with reading 
The json file loaded uses distances with a threshold set at 0.01, a standard threhsold used often in connectome predictive modeling

In [12]:
### distances
with open("positive_distances.json") as json_file:
    positiveDistances = json.load(json_file)

with open("negativeDistances.json") as json_file:
    negativeDistances = json.load(json_file)
### FC edges
with open("positiveFC.json") as json_file:
    positiveFC = json.load(json_file)

with open("negativeFC.json") as json_file:
    negativeFC = json.load(json_file)

### Train the model

In [13]:
def clean_data(subjectList, all_confs, all_SM, all_dist, mlab=eng):
    ### remove confounds for a group of subjects
    ### always done independently to avoid leakage

    ## set the confounds
    confs = all_confs.loc[subjectList]
    confs = set_confounds(confs, mlab)
    ## regress them from behavioral measures
    behavior = all_SM.loc[subjectList]
    behavior = preprocess_SM(behavior, confs, mlab)
    ## regress them from distance measures
    distance = all_dist.loc[subjectList]
    distance = preprocessDists(distance, confs)

    return confs, behavior, distance

In [14]:
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [15]:
tasks = "ReadEng_Unadj"  ### this notebook runs on reading.

This is the most basic version of CPM

In [16]:
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [17]:
def prepare_summary_measures(task, edges, Dist, ablate=False):
    """Prepares summary measures for analysis based on edges.

    Parameters:
    task (str): The specific task for which edges are being analyzed.
    edges (dict or list): A dictionary of edges for multiple tasks or a list of edge lists (positive and negative).
    Dist (pd.DataFrame): A DataFrame containing distance measures for edges.
    ablate (list, optional): A list of edges to include in the summary measure. Defaults to False.

    Returns:
    pd.DataFrame: A DataFrame containing the summary measure for the specified task.
    """

    # Check if edges is a dictionary (single task mode) or a list (positive and negative edges)
    single_status = isinstance(edges, dict)
    if single_status:
        edge_list = edges[task]
        n_edges = len(edge_list)
        if ablate:
            edge_list = [j for j in edge_list if j in ablate]

    else:
        edge_list = {"pos": edges[0][task], "neg": edges[1][task]}
        n_edges = len(edge_list["pos"]) + len(edge_list["neg"])

        if ablate:
            # Filter edges based on ablate list
            pos = [j for j in edge_list["pos"] if j in ablate]
            neg = [j for j in edge_list["neg"] if j in ablate]

            edge_list = {"pos": pos, "neg": neg}

            # Logging included edges
            if pos:
                print(f"{len(pos)} positive edges included")
            if neg:
                print(f"{len(neg)} negative edges included")

    # Calculate the summary measure based on the edge list
    #     print(edge_list)
    if single_status:
        summary_measure = pd.DataFrame(Dist[edge_list].sum(axis=1))
        print(f"{len(edge_list)}/{n_edges} edges used in summary metric")
    else:
        p_n_Edge = len(edge_list["pos"])
        n_n_Edge = len(edge_list["neg"])
        print(f"{p_n_Edge + n_n_Edge}/{n_edges} edges used in summary metric")

        if p_n_Edge > 0 and n_n_Edge == 0:
            summary_measure = pd.DataFrame(Dist[edge_list["pos"]].sum(axis=1))
        elif p_n_Edge == 0 and n_n_Edge > 0:
            summary_measure = pd.DataFrame(Dist[edge_list["neg"]].sum(axis=1))
        else:
            summary_measure = pd.DataFrame(
                [Dist[edge_list["pos"]].sum(axis=1), Dist[edge_list["neg"]].sum(axis=1)]
            ).T

    return summary_measure

In [18]:
from sklearn.metrics import mean_squared_error


def clean_folds(task, cv_folds, confounds, Beh, SummaryMeasures, mlab=eng):
    training_dist = {}
    training_beh = {}

    val_dist = {}
    val_beh = {}
    for fold in cv_folds:
        #### get the subject ID's for each fold
        training_idx = cv_folds[fold]["training"]
        val_idx = cv_folds[fold]["testing"]
        ### extract confounds
        ### get confounds training set
        confs_train = confounds.loc[training_idx]
        confs_train = set_confounds(confs_train, mlab)
        ### get confounds validation
        confs_val = confounds.loc[val_idx]
        confs_val = set_confounds(confs_val, mlab)
        ### clean behavioral variables
        #         print('cleaning behavioral data')
        beh_train = preprocess_SM(Beh.loc[training_idx], confs_train, mlab)[task]

        beh_val = preprocess_SM(Beh.loc[val_idx], confs_val, mlab)[
            task
        ]  ### the valiadtoin data remains untouched
        ### clean distance data
        #         print('training task dist models')
        train_d = SummaryMeasures.loc[training_idx]
        train_d = preprocessDists(train_d, confs_train)
        ### extract validatoin set data
        val_d = SummaryMeasures.loc[val_idx]
        val_d = preprocessDists(val_d, confs_val)

        training_beh[fold] = beh_train
        training_dist[fold] = train_d
        val_dist[fold] = val_d
        val_beh[fold] = beh_val

    return training_dist, training_beh, val_dist, val_beh


def train_test_predict(trainX, trainY, valX, valY):
    # Set up the pipeline with StandardScaler and LinearRegression
    lr = Pipeline(
        [
            ("scaler", StandardScaler()),  # Scale features
            ("reg", LinearRegression(fit_intercept=True)),  # Apply linear regression
        ]
    )

    # No need to reshape inside the function; let's handle outside if needed
    # Fit the model
    lr.fit(trainX, trainY)

    # Predict using the model
    predY = lr.predict(valX)
    r, p = pearsonr(predY.squeeze(), valY)

    # Extract coefficients and intercept
    coefficients = lr.named_steps["reg"].coef_
    intercept = lr.named_steps["reg"].intercept_
    mse = mean_squared_error(valY, predY)

    return r, coefficients, intercept, mse


def fit_and_measure(training_dist, training_beh, val_dist, val_beh, skip_perms=False):
    corr_dict = {}
    beta_dict = {}
    inter_dict = {}
    mse_dict = {}

    for fold in training_dist.keys():
        perm_file_train = f"fold_permutations/training/{fold}.csv"
        train_perms = pd.read_csv(perm_file_train, header=None)
        perm_file_val = f"fold_permutations/validation/{fold}.csv"
        val_perms = pd.read_csv(perm_file_val, header=None)

        corr_vals = []
        beta_vals = []
        intercepts = []
        mse_vals = []
        ### do the test and perms for this fold
        if skip_perms == False:
            for perm_idx in range(train_perms.shape[1]):
                t_idx = train_perms[perm_idx].values
                v_idx = val_perms[perm_idx].values
                r, beta, inter, mse = train_test_predict(
                    training_dist[fold],
                    training_beh[fold].iloc[t_idx],
                    val_dist[fold],
                    val_beh[fold],
                )
                corr_vals.append(r)
                beta_vals.append(beta)
                intercepts.append(inter)
                mse_vals.append(mse)
        else:
            #             print('skipping permutation tests')
            t_idx = train_perms[0].values
            v_idx = val_perms[0].values
            r, beta, inter, mse = train_test_predict(
                training_dist[fold],
                training_beh[fold].iloc[t_idx],
                val_dist[fold],
                val_beh[fold],
            )
            corr_vals.append(r)
            beta_vals.append(beta)
            intercepts.append(inter)
            mse_vals.append(mse)
        corr_dict[fold] = corr_vals
        beta_dict[fold] = beta_vals
        inter_dict[fold] = intercepts
        mse_dict[fold] = mse_vals
    return corr_dict, beta_dict, inter_dict, mse_dict

### a few plotting functions as well

In [19]:
def PlotModalityPerformance(ref_data, ablated_data, multimodal_model):
    sns.set_theme(style="whitegrid")
    sns.set_context("talk", font_scale=0.7)  # Font scale reduced for small font sizes
    #     plt.rcParams['font.family'] = 'Arial'

    # Prepare data
    ref_melted = ref_data.reset_index().melt(
        id_vars="index", var_name="Fold", value_name="Value"
    )
    ref_melted["Type"] = "CD"
    ref_melted["Model"] = [f"CD-{i}" for i in ref_melted["index"]]

    ablated_melted = ablated_data.reset_index().melt(
        id_vars="index", var_name="Fold", value_name="Value"
    )
    ablated_melted["Type"] = "FC"
    ablated_melted["Model"] = [f"FC-{i}" for i in ablated_melted["index"]]

    combined_data = pd.concat([ref_melted, ablated_melted])
    combined_data["Type"] = pd.Categorical(
        combined_data["Type"], categories=["FC", "CD"], ordered=True
    )

    multimodal_model = multimodal_model.melt(var_name="Fold", value_name="Value")
    multimodal_model["index"] = "Joint"
    multimodal_model["Type"] = "FC+CD"
    multimodal_model["Model"] = "MultiModal-Joint"
    multimodal_model = multimodal_model[["index", "Fold", "Value", "Type", "Model"]]
    joint_data = combined_data[combined_data["index"] == "Joint"]
    joint_data = pd.concat([multimodal_model, joint_data])
    joint_data["Type"] = pd.Categorical(
        joint_data["Type"], categories=["FC", "CD", "FC+CD"], ordered=True
    )

    # Define color palettes
    palette_joint = {"FC": "#8a2be2", "CD": "#6a0dad", "FC+CD": "#008080"}
    palette_pos = {"FC": "#ff6347", "CD": "#ff4500"}
    palette_neg = {"FC": "#4682b4", "CD": "#1e90ff"}

    # Create subplot layout with constrained figure size
    fig, axes = plt.subplots(
        1,
        3,
        figsize=(4.72, 1.8),
        gridspec_kw={"width_ratios": [1.2, 1.2, 1.6]},
        sharey=True,
    )

    # Generate Violin and Box plots
    for ax, data, palette, title in zip(
        axes,
        [
            combined_data[combined_data["index"] == "Positive"],
            combined_data[combined_data["index"] == "Negative"],
            joint_data,
        ],
        [palette_pos, palette_neg, palette_joint],
        ["Positive", "Negative", "Joint"],
    ):
        sns.violinplot(
            x="Type",
            y="Value",
            hue="Type",
            legend=False,
            data=data,
            ax=ax,
            palette=palette,
            inner="box",
            linewidth=1.5,
        )
        ax.set_title(title, fontsize=7)

    # Style adjustments
    for ax in axes:
        ax.set_xlabel("")
        ax.set_ylabel("")
        ax.tick_params(axis="x", labelsize=7)
        ax.tick_params(axis="y", labelsize=7)
        sns.despine(ax=ax, top=True, right=True, left=True, bottom=False)
        ax.grid(True, linestyle="--", alpha=0.5, linewidth=0.5)

    axes[0].set_ylabel("Correlation (r)", fontsize=7)

    plt.tight_layout()
    plt.subplots_adjust(wspace=0.2)

### Run single modality and joint modality models -- get performance

In [20]:
dist_summary_measures = prepare_summary_measures(
    "ReadEng_Unadj", [positiveDistances, negativeDistances], SchaeferDist, ablate=False
)
FC_summary_measures = prepare_summary_measures(
    "ReadEng_Unadj", [positiveFC, negativeFC], SchaeferFC, ablate=False
)

476/476 edges used in summary metric
157/157 edges used in summary metric


In [21]:
training_dist, training_beh, val_dist, val_beh = clean_folds(
    "ReadEng_Unadj", folds, conf_categories, subjSM, dist_summary_measures
)

training_FC, training_beh, val_FC, val_beh = clean_folds(
    "ReadEng_Unadj", folds, conf_categories, subjSM, FC_summary_measures
)
## cleaning of the below 4 column set was quite different than cleaning apart
multimodal = (
    pd.concat([dist_summary_measures, FC_summary_measures], axis=1)
    .T.reset_index()
    .drop("index", axis=1)
    .T
)
TrainMMcleanAll, training_beh, valMMcleanAll, val_beh = clean_folds(
    "ReadEng_Unadj", folds, conf_categories, subjSM, multimodal
)

In [ ]:
### distance
corrsDist, betasDist, intersDist, mseDist = fit_and_measure(
    training_dist, training_beh, val_dist, val_beh, skip_perms=False
)
corrsDist = pd.DataFrame(corrsDist)
print(f"Mean prediction: {corrsDist.mean(axis=1)[0]}")
pvalDist = sum(corrsDist.mean(axis=1)[0] <= corrsDist.iloc[1:].mean(axis=1)) / len(
    corrsDist
)
print(f"pval: {pvalDist}")

In [ ]:
### fc
corrsFC, betasFC, intersFC, mseFC = fit_and_measure(
    training_FC, training_beh, val_FC, val_beh, skip_perms=False
)
corrsFC = pd.DataFrame(corrsFC)
print(f"Mean prediction: {corrsFC.mean(axis=1)[0]}")
pvalFC = sum(corrsFC.mean(axis=1)[0] <= corrsFC.iloc[1:].mean(axis=1)) / len(corrsFC)
print(f"pval: {pvalFC}")

In [ ]:
### multimodal dist + fc
corrsMM, betasMM, intersMM, mseMM = fit_and_measure(
    TrainMMcleanAll, training_beh, valMMcleanAll, val_beh, skip_perms=False
)
corrsMM = pd.DataFrame(corrsMM)
print(f"Mean prediction: {corrsMM.mean(axis=1)[0]}")
pvalMM = sum(corrsMM.mean(axis=1)[0] <= corrsMM.iloc[1:].mean(axis=1)) / len(corrsMM)
print(f"pval: {pvalMM}")

### Run Positive and negative single feature models for FC and Dist

In [ ]:
def run_full_analysis(
    task, cv_folds, confounds, Beh, Dist, edges, skip_perms=False, ablation=False
):
    summary_measure = prepare_summary_measures(task, edges, Dist, ablate=ablation)
    #     print('running')
    ### clean all the data at once
    training_dist, training_beh, val_dist, val_beh = clean_folds(
        task, cv_folds, confounds, Beh, summary_measure
    )

    corrs, betas, inters, mse = fit_and_measure(
        training_dist, training_beh, val_dist, val_beh, skip_perms=skip_perms
    )

    corr_df = pd.DataFrame(corrs)
    print(f"Mean performance: {corr_df.loc[0].mean()}")
    if skip_perms == False:
        Pval = sum(corr_df.mean(axis=1)[0] <= corr_df.iloc[1:].mean(axis=1)) / len(
            corr_df
        )
        print(f"pval: {Pval} \n")

    return corrs, betas, inters, mse

In [ ]:
### distance
print("Positive Distance Model")
posModelDist, pos_betasDist, _, _ = run_full_analysis(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferDist,
    positiveDistances,
    skip_perms=False,
    ablation=False,
)
posModelDist = pd.DataFrame(posModelDist)
print("")
print("Negative Distance Model")
negModelDist, neg_betasDist, _, _ = run_full_analysis(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferDist,
    negativeDistances,
    skip_perms=False,
    ablation=False,
)
negModelDist = pd.DataFrame(negModelDist)
print("\n")
### FC
print("Positive FC Model")
posModelFC, pos_betasFC, _, _ = run_full_analysis(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferFC,
    positiveFC,
    skip_perms=False,
    ablation=False,
)
posModelFC = pd.DataFrame(posModelFC)
print("")
print("Negative FC Model")
negModelFC, neg_betasFC, _, _ = run_full_analysis(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferFC,
    negativeFC,
    skip_perms=False,
    ablation=False,
)
negModelFC = pd.DataFrame(negModelFC)

In [ ]:
dist = pd.concat([corrsDist.loc[0], posModelDist.loc[0], negModelDist.loc[0]], axis=1)
dist.columns = ["Joint", "Positive", "Negative"]
dist = dist.T
func = pd.concat([corrsFC.loc[0], posModelFC.loc[0], negModelFC.loc[0]], axis=1)
func.columns = ["Joint", "Positive", "Negative"]
func = func.T
gd_and_fc = pd.DataFrame(corrsMM.loc[0])

dist.to_csv("distance_CV_results.csv")

In [ ]:
PlotModalityPerformance(dist, func, gd_and_fc)

### Are the models statistically different? 

In [ ]:
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

In [ ]:
### pos and neg FC vs Dsit
wilcx_pos = wilcoxon(func.loc["Positive"], dist.loc["Positive"])
wilcx_neg = wilcoxon(func.loc["Negative"], dist.loc["Negative"])

### joint model testing
wilcx_jnt = wilcoxon(func.loc["Joint"], dist.loc["Joint"])
wilcx_jnt_MM_dist = wilcoxon(dist.loc["Joint"], gd_and_fc.values.squeeze())
wilcx_jnt_MM_FC = wilcoxon(dist.loc["Joint"], gd_and_fc.values.squeeze())
### look at the p-vals and correct them too
p_vals = [
    wilcx_pos[1],
    wilcx_neg[1],
    wilcx_jnt[1],
    wilcx_jnt_MM_FC[1],
    wilcx_jnt_MM_dist[1],
]
corrected_p_vals = multipletests(p_vals, method="bonferroni")[1]
summary_df = pd.DataFrame(
    [p_vals, corrected_p_vals],
    columns=["Positive", "Negative", "Joint", "FC+CD vs Func", "FC+CD vs Dist"],
    index=["p_vals", "corrected p"],
)
summary_df

We see that the FC and CD Joint feature models don't pass significance. 
We test now for equivalence of the two joint models using two one sided wilcoxon tests

In [ ]:
#### here is the code for equivalence testing.
#### equivalence testing set up


def bootstrap_ci(data, n_bootstrap=10000, ci=95, seed=42):
    """
    Calculates the confidence interval for the mean of a dataset using bootstrapping.

    Args:
        data (array-like): The dataset from which to calculate the mean.
        n_bootstrap (int): The number of bootstrap samples to generate.
        ci (int): The desired confidence level (e.g., 95 for 95% CI).
        seed (int): Seed for the random number generator for reproducibility.

    Returns:
        tuple: Lower and upper bounds of the confidence interval.
    """
    np.random.seed(seed)  # Set seed for reproducibility
    sample_size = len(data)
    boot_means = []

    # Generate bootstrap samples and compute the mean of each sample
    for _ in range(n_bootstrap):
        boot_sample = np.random.choice(data, size=sample_size, replace=True)
        boot_mean = np.mean(boot_sample)
        boot_means.append(boot_mean)

    # Calculate the confidence interval
    lower_bound = np.percentile(boot_means, (100 - ci) / 2)
    upper_bound = np.percentile(boot_means, 100 - (100 - ci) / 2)
    #     print(lower_bound,upper_bound,np.mean(data))

    return upper_bound - lower_bound


#### we can use the confidence interval to set the bounds for the tost.
### if it is above 0.03 then we'll set to 0.03 to keep tight bounds


def wilcoxon_tost(data1, data2, bound=0.03):
    """
    Performs a Two One-Sided Tests (TOST) for equivalence using the Wilcoxon signed-rank test.

    Args:
        data1 (array-like): First dataset, typically predictions or measurements from one model.
        data2 (array-like): Second dataset, typically predictions or measurements from another model.
        bound (float): The equivalence margin or bound within which differences are considered practically equivalent.

    Returns:
        float: The maximum p-value from the two one-sided tests. A small value (< alpha level, typically 0.05)
               suggests that the difference between data1 and data2 is not practically significant.
    """
    # Convert to arrays and calculate differences
    diffs = data1 - data2

    # Perform TOST using Wilcoxon signed-rank tests
    p_lower = wilcoxon(diffs - bound, alternative="less", correction=True).pvalue
    p_upper = wilcoxon(diffs + bound, alternative="greater", correction=True).pvalue

    # Return the maximum p-value to apply the TOST principle
    return max(p_lower, p_upper)

In [ ]:
p_equiv_joint = wilcoxon_tost(dist.loc["Joint"], func.loc["Joint"])
print(
    f"The p-value of the equivalence test between the CD and FC models is \n {p_equiv_joint}"
)

The p-values imply that the Joint FC and CD models are neither equivalent, nor statistically different. 

They're somewhere in between

## Ablation of summary features in the distance models
To determine what distances matter in our models we'll now ablate the summary features and retrain and evaluate with CV. We will not run permutations here as we're only interested in how including certain sets of distances effects predictive accuracy in the joint distance model

In [ ]:
def ModelAblation(
    tasks, folds, conf_categories, subjSM, SchaeferDist, Clusters, Distances
):
    """
    Performs analysis on provided clusters and calculates correlations, p-values, and MSE.

    Parameters:
    - tasks: Information about tasks (global variable).
    - folds: Number of folds for cross-validation (global variable).
    - conf_categories: Configuration categories (global variable).
    - subjSM: Subject similarity matrix (global variable).
    - SchaeferDist: Schaefer Distance Matrix (global variable).
    - LnegClusters: Dictionary of negative clusters.
    - negativeDistances: Negative distance matrix.

    Returns:
    - Tuple of dictionaries containing correlations, p-values, mean squared errors, and MSE for each cluster.
    """
    L_corrs = {}
    L_r = {}
    #     L_p = {}
    L_mse = {}
    L_betas = {}
    for cluster in Clusters:
        print(f"using only {cluster}")

        ablated, betas, _, mse = run_full_analysis(
            tasks,
            folds,
            conf_categories,
            subjSM,
            SchaeferDist,
            Distances,
            skip_perms=True,
            ablation=Clusters[cluster],
        )

        # Calculate correlations and other statistics
        L_betas[cluster] = betas
        df_ablated = pd.DataFrame(ablated)
        L_corrs[cluster] = df_ablated.iloc[0]
        ablated_mean = df_ablated.mean(axis=1).iloc[0]
        #         p_value = (df_ablated.mean(axis=1).iloc[0] <= df_ablated.mean(axis=1).iloc[1:]).sum() / len(df_ablated.mean(axis=1))

        # Print cluster information
        #         print(f'cluster {cluster}')
        print(f"")

        # Store results in dictionaries
        L_r[cluster] = ablated_mean
        #         L_p[cluster] = p_value
        L_mse[cluster] = mse

    return L_corrs, L_r, L_mse, L_betas


#### function to get list of distances touching a given Yeo network in Schaefer atlas
def rsn_ablation_lists(dists):
    redes = ["Vis", "SomMot", "DorsAttn", "SalVentAttn", "Limbic", "Cont", "Default"]

    nwork_dists = {}
    for red in redes:
        nwork_dists[f"Network: {red}"] = [i for i in dists if red in i]
    return nwork_dists

In [ ]:
### set up the network specific distances
reading_dist_neg = negativeDistances["ReadEng_Unadj"]
rsns_neg_dists = rsn_ablation_lists(reading_dist_neg)
reading_dist_pos = positiveDistances["ReadEng_Unadj"]
rsns_pos_dists = rsn_ablation_lists(reading_dist_pos)
joint_rns = {}
for i, j in zip(rsns_neg_dists, rsns_pos_dists):
    joint_rns[i] = rsns_neg_dists[i] + rsns_pos_dists[j]

In [ ]:
### run the network specific distance models
corrsJNT_rsn, _, _, _ = ModelAblation(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferDist,
    joint_rns,
    [positiveDistances, negativeDistances],
)

In [ ]:
corrsJNT_rsn = pd.DataFrame(corrsJNT_rsn)
corrsJNT_rsn.to_csv("./network_centered_CV_results.csv")

## Get pairwise distance sets between RSNs

In [ ]:
def rsn_ablation_lists(pos_dist, neg_dist):
    redes = ["Vis", "SomMot", "DorsAttn", "SalVentAttn", "Limbic", "Cont", "Default"]
    inclusion = {}
    for red in redes:
        red_pos = [i for i in pos_dist if red in i]
        red_neg = [i for i in neg_dist if red in i]
        nw_dist = red_pos + red_neg

        for subred in redes:
            sr_dist = [i for i in nw_dist if subred in i]
            if subred == red:
                within_nw = []
                for i in nw_dist:
                    edge = i.split("--")
                    node1 = edge[0]
                    node2 = edge[1]
                    if red in node1 and red in node2:
                        within_nw.append(i)
                sr_dist = within_nw

            inclusion[f"{red}--{subred}"] = sr_dist

    return inclusion

In [ ]:
pairwise_ablations = rsn_ablation_lists(reading_dist_pos, reading_dist_neg)

In [ ]:
### note this can take a while. saved results can be loaded
corrsJNT_pairwise_ablated, _, _, _pairwise_betas = ModelAblation(
    tasks,
    folds,
    conf_categories,
    subjSM,
    SchaeferDist,
    pairwise_ablations,
    [positiveDistances, negativeDistances],
)

In [ ]:
internetwork_models = pd.DataFrame(corrsJNT_pairwise_ablated)
internetwork_models.to_csv("internetwork_models_CV_results.csv")

### Visualize ablated models and test equivalence among them

In [ ]:
#### plotting function
import matplotlib.colors


def plot_performance(
    data, x_col, y_col, figsize_mm=(175, 110), rotation_angle=45, col_range=None
):
    # Define specific color mappings
    palette = {
        "Full Model": "#60C2A5",
        "Vis": "#a252ad",
        "SM": "#789ac0",
        "Lim": "#f6fdc9",
        "SAN": "#e065fe",
        "DAN": "#409832",
        "Cont": "#f0b943",
        "DMN": "#d8717c",
    }

    # Calculate the mean of 'r' for each 'CD Included' category
    means = data.groupby(x_col)[y_col].mean().reset_index()
    # Sort the means
    sorted_means = means.sort_values(y_col, ascending=False)

    # Use the sorted order of 'CD Included' for plotting, potentially limited by col_range
    if col_range and isinstance(col_range, int) and col_range < len(sorted_means):
        sorted_categories = sorted_means[x_col].head(col_range).tolist()
    else:
        sorted_categories = sorted_means[x_col].tolist()

    # Extend the palette to include all categories in your data, assigning a default neutral grey with transparency
    all_categories = data[x_col].unique()
    uniform_color = matplotlib.colors.to_rgba("#e6f2ee", alpha=0.5)
    palette.update({cat: uniform_color for cat in all_categories if cat not in palette})

    # Set the categories with sorted order and defined colors
    data[x_col] = pd.Categorical(
        data[x_col], categories=sorted_categories, ordered=True
    )

    # Convert mm to inches for dimensions
    width_in_inches = figsize_mm[0] / 25.4  # Convert mm to inches
    height_in_inches = figsize_mm[1] / 25.4  # Convert mm to inches

    # Set up the matplotlib figure with high DPI for publication quality
    sns.set(style="whitegrid")
    plt.figure(figsize=(width_in_inches, height_in_inches), dpi=300)

    ax = sns.boxplot(
        data=data,
        x=x_col,
        y=y_col,
        hue=x_col,
        legend=False,
        width=0.75,
        palette=palette,
    )

    # Remove titles and labels (no overarching title, no x or y axis titles)
    ax.set_title("")
    ax.set_xlabel("")
    ax.set_ylabel("")

    # Despine for a cleaner look
    sns.despine(ax=ax, top=True, right=True, left=True, bottom=True)

    # Adjust tick parameters for better visibility
    ax.tick_params(axis="both", which="major", labelsize=7)

    # Rotate x labels for better fit and readability
    ax.set_xticklabels(
        ax.get_xticklabels(), rotation=rotation_angle, ha="center", fontsize=5
    )

    # Ensure layout fits the constraints and is tight
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
network_centered = corrsJNT_rsn.copy()
network_centered.rename(
    columns={
        "Network: SomMot": "SM",
        "Network: DorsAttn": "DAN",
        "Network: SalVentAttn": "SAN",
        "Network: Limbic": "Lim",
        "Network: Default": "DMN",
        "Network: Vis": "Vis",
        "Network: Cont": "Cont",
    },
    inplace=True,
)

In [ ]:
internetwork__performance = internetwork_models.mean()
internetwork__performance.loc["Limbic--Limbic"] = (
    np.nan
)  ### because there are no limbic to limbics
internetwork__performance = pd.DataFrame(internetwork__performance)

In [ ]:
def reformat_entries(network_list, abbrev_dict):
    reformatted_list = []
    for entry in network_list:
        # Split the entry at '--'
        parts = entry.split("--")
        # Replace each part if it is in the abbreviation dictionary
        reformatted_parts = [abbrev_dict.get(part, part) for part in parts]
        # Join the parts back together and add to the new list
        reformatted_list.append("--".join(reformatted_parts))
    return reformatted_list

In [ ]:
### let's remove repeats from the internetwork columns
dists = internetwork_models.columns
unique_dists = set()

for in_dist in dists:
    part1, part2 = in_dist.split("--")
    # Store the sorted tuple (to handle symmetry) in the set
    unique_dists.add(tuple(sorted([part1, part2])))

# Convert the tuples back to the original format
unique_internetworks = ["--".join(pair) for pair in unique_dists]
abbreviation_dict = {
    "SomMot": "SM",
    "DorsAttn": "DAN",
    "SalVentAttn": "SAN",
    "Limbic": "Lim",
    "Default": "DMN",
}
unique_internetworks_new = reformat_entries(unique_internetworks, abbreviation_dict)

In [ ]:
inter2plot = internetwork_models[unique_internetworks].rename(
    columns=dict(zip(unique_internetworks, unique_internetworks_new))
)

In [ ]:
#### let's get all model performance into one df
all_joint_models = pd.concat([dist.loc["Joint"], network_centered, inter2plot], axis=1)
all_joint_models.rename(columns={"Joint": "Full Model"}, inplace=True)
### drop the limbic--limbic there were not distances at all here so it ended up using the full model there. it can thus be ignored
all_joint_models.drop(columns="Lim--Lim", inplace=True)

### sort by mean performance
all_joint_models = all_joint_models[
    all_joint_models.mean(axis=0).sort_values(ascending=False).keys()
]

In [ ]:
sns.set_context("notebook", font_scale=0.5, rc={"lines.linewidth": 2.5})

In [ ]:
plot_performance(
    data=all_joint_models.melt(var_name="CD Included", value_name="r"),
    x_col="CD Included",
    y_col="r",
    figsize_mm=(88, 70),
    col_range=5,
    rotation_angle=0,
)

In [ ]:
plot_performance(
    data=all_joint_models.melt(var_name="CD Included", value_name="r"),
    x_col="CD Included",
    y_col="r",
    figsize_mm=(88, 70),
    rotation_angle=90,
)

### Let's check equivalence among all models

In [ ]:
from tqdm import tqdm

In [ ]:
### test pairwise equivalence of all 35 models
def tost_a_df(df):
    results_dict = {}
    bounds_dict = {}
    for i in tqdm(df.keys()):
        ref_bounds = bootstrap_ci(df[i])
        for j in df.keys():
            if i == j or (j, i) in results_dict.keys():
                pass
            else:
                pair_bounds = bootstrap_ci(df[j])
                bound = min(ref_bounds, pair_bounds)

                if bound > 0.03:
                    bound = 0.03
                p = wilcoxon_tost(df[i], df[j], bound=bound)
                results_dict[(i, j)] = p
                bounds_dict[(i, j)] = bound
    return results_dict, bounds_dict

In [ ]:
model_equiv = tost_a_df(all_joint_models)

In [ ]:
model_equiv_df = pd.DataFrame([model_equiv[0]])
corrections = multipletests(
    model_equiv_df.values.squeeze(), method="fdr_bh", alpha=0.05
)
passed_tests = np.where(corrections[0])[0]

In [ ]:
models_passing = model_equiv_df.iloc[:, passed_tests]
models_passing.index = ["p_raw"]
models_passing = models_passing.T
models_passing["p_fdr"] = corrections[1][corrections[0]]

In [ ]:
models_passing

### Finished
Continue to netbook 4 for feature visualization